In [ ]:
import pandas as pd
import spacy

nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [ ]:
df = pd.read_csv("bertowy_model_z_food_med.csv")
df.head()

In [ ]:
def repl(data):
    data = data.split(',')
    for i in range(len(data)):
        data[i] = data[i].replace("[", "")
        data[i] = data[i].replace("]", "")
        data[i] = data[i].replace("'", "")
        data[i] = data[i].replace('"', "")
        data[i] = data[i].strip()
    return data

def update_list(data, idd):
    for i in idd:
        data['food'][i] = list(set(data['food'][i]))
        data['count_f'][i] = len(data['food'][i])
        data['med'][i] = list(set(data['med'][i]))
        data['count_m'][i] = len(data['med'][i])
    return data

In [ ]:
good_df = df[(df['count_f'] != 0) & (df['count_m'] != 0)]
numb_id = good_df.index.tolist()

for i in numb_id:
    good_df['med'][i] = repl(good_df['med'][i])
    good_df['food'][i] = repl(good_df['food'][i])

update_list(good_df, numb_id)

In [ ]:
good_df.to_csv('bertowy_model_med_food_together.csv')

In [ ]:
def get_pairs(data):

    list_food = []

    for i in numb_id:
        for j in range(len(data['food'][i])):
            for k in range(len(data['med'][i])):
                list_food.append([data['food'][i][j],data['med'][i][k]])

   
    return list_food

def get_relation(sent):

  doc = nlp(sent)
  matcher = Matcher(nlp.vocab)


  pattern = [[{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}]] 


  matcher.add("matching_1", None, *pattern) 
  matches = matcher(doc)
  k = len(matches) - 1
  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

def multi_rel(rel, data, id):
    relations = []
    abst_id = []
    ff = []
    for j in id: 
        ff.append(data['count_f'][j] * data['count_m'][j])


    for i in range(len(rel)):
        for k in range(ff[i]):
            relations.append(rel[i])


    return relations

def get_id(data, id):
    count = 0
    list_id = []
    for j in id:
        count = (data['count_f'][j] * data['count_m'][j])
        for i in range(count):
            list_id.append(data['id'][j])

    return list_id

In [ ]:
ent_pairs = get_pairs(good_df)
relations = [get_relation(i) for i in tqdm(good_df['text'])]
relations = multi_rel(relations, good_df, numb_id)
id_numb_list = get_id(good_df, numb_id)

In [ ]:
source = [i[0] for i in ent_pairs]
target = [i[1] for i in ent_pairs]

triples_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations, 'id': id_numb_list})

triples_df.to_csv('spacy_final_result.csv', index = False)

In [ ]:
def show_triples(choose, word, numb):
    if choose == 1:
        print(triples_df[triples_df['edge'] == word][:numb])
    if choose == 2:
        print(triples_df[triples_df['source'] == word][:numb])
    if choose == 3:
        print(triples_df[triples_df['target'] == word][:numb])    
    
def make_graph_relation(word):
    G=nx.from_pandas_edgelist(triples_df[triples_df['edge']==word], "source", "target", 
                            edge_attr=True, create_using=nx.MultiDiGraph())

    plt.figure(figsize=(15,15))
    pos = nx.spring_layout(G, k = 0.5)
    nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
    plt.show()

In [ ]:
# 1 - relation, 2 - food, 3 - med
show_triples(3, 'cancer', 20)

In [ ]:
make_graph_relation("associated")